In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r drive/My\ Drive/Bin/TechJam2019/* .

In [0]:
!cp -r logs/* drive/My\ Drive/Bin/TechJam2019/logs

In [1]:
!nvidia-smi 

Thu Nov  7 15:59:10 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 430.50       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 28%   26C    P8     3W / 250W |  10925MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:0B:00.0 Off |                  N/A |
| 28%   31C    P5    24W / 250W |    165MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, GRU, CuDNNLSTM, CuDNNGRU, Bidirectional, Masking
from keras.layers import Dense, InputLayer, Dropout, BatchNormalization
from keras.layers import Reshape, Flatten, Conv1D, AveragePooling1D, MaxPool1D, Activation, Concatenate
from keras.layers import Conv1D
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
import keras.backend as K
from sklearn.model_selection import train_test_split

from tqdm import tqdm, trange, tnrange
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [23]:
from data_gen import create_weekly_kplus, create_credit_seq, create_daily_kplus, create_demographic_data, create_weekly_credit
from util import modified_SMAPE
from losses import SMAPE_loss
from callbacks import EvaluateSMAPE

load data gen


In [6]:
demograpgics = pd.read_csv('datasets/exam-1/demographics.csv')
cc = pd.read_csv('datasets/exam-1/cc.csv')
cc.sort_values(by=['cc_no', 'pos_dt'], inplace=True)
kplus = pd.read_csv('datasets/exam-1/kplus.csv')
kplus.sort_values(by=['id', 'sunday'], inplace=True)
train_set = pd.read_csv('datasets/exam-1/train.csv')
test_set = pd.read_csv('datasets/exam-1/test.csv')
raw_demographics = pd.read_csv('datasets/exam-1/raw_demograhgics.csv')

In [7]:
scaled_kplus = kplus.copy()
kplus_scaler =  StandardScaler()
scaled_kplus[['kp_txn_count', 'kp_txn_amt']] = kplus_scaler.fit_transform(kplus[['kp_txn_count', 'kp_txn_amt']])

In [8]:
cc_persons = pd.read_csv('datasets/exam-1/cc_persons.csv')
cc_scaler = StandardScaler()
cc_persons[['cc_txn_amt', 'count']] = cc_scaler.fit_transform(cc_persons[['cc_txn_amt', 'count']])

In [9]:
cc_weekly = pd.read_csv('datasets/exam-1/cc_weekly_persons.csv')
cc_weekly_scaler = StandardScaler()
cc_weekly[['cc_txn_amt', 'count']] = cc_weekly_scaler.fit_transform(cc_weekly[['cc_txn_amt', 'count']])

In [10]:
train_scaler = StandardScaler()
scaled_train_set = train_set.copy()
scaled_train_set.set_index('id', inplace=True)
scaled_train_set['income'] = train_scaler.fit_transform(np.expand_dims(scaled_train_set['income'], axis=2))

/home/porlolicon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


# KPLUS

In [0]:
train_kplus = train_kplus[train_kplus['id'] <= 50000]

In [0]:
train_kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt
822506,1,2018-06-03,-0.567504,-0.211370
822505,1,2018-06-17,-0.567504,-0.207460
100625,2,2018-01-07,-0.399415,-0.218408
100629,2,2018-01-14,-0.231325,-0.198075
100641,2,2018-01-21,-0.399415,-0.218408


In [0]:
padding_value = float(-100)
xs = create_weekly_kplus(train_kplus, padding_value)
ys = scaled_train_set.loc[train_kplus['id'].drop_duplicates().to_numpy()].to_numpy()

In [0]:
base_activation = 'tanh'
model = Sequential()
model.add(InputLayer((xs.shape[1], 2)))
model.add(Masking(padding_value, ))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation))
model.add(Dense(1, activation='linear'))

In [0]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [0]:
model.compile(loss=SMAPE_loss(kplus_scaler), optimizer=opt )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 25, 2)             0         
_________________________________________________________________
masking_2 (Masking)          (None, 25, 2)             0         
_________________________________________________________________
gru_6 (GRU)                  (None, 25, 50)            7950      
_________________________________________________________________
gru_7 (GRU)                  (None, 25, 50)            15150     
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 50)            0         
_________________________________________________________________
gru_8 (GRU)                  (None, 25, 25)            5700      
_________________________________________________________________
gru_9 (GRU)                  (None, 25, 25)            3825      
__________

In [0]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [0]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=kplus_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir='logs/gru')],
          initial_epoch=0,
         epochs=10000)

Instructions for updating:
Use tf.cast instead.
Train on 27725 samples, validate on 6932 samples
Epoch 1/10000
27725/27725 [==============================] - 58s 2ms/step - loss: 0.0896 - val_loss: 0.0827
Epoch 1 | val-SMAPE: 91.72752684189692
Epoch 2/10000
27725/27725 [==============================] - 71s 3ms/step - loss: 0.0849 - val_loss: 0.0796
Epoch 2 | val-SMAPE: 92.04023577019237
Epoch 3/10000
 4256/27725 [===>..........................] - ETA: 52s - loss: 0.0854

KeyboardInterrupt: 

# Credit

In [10]:
cc_persons.head()

,Unnamed: 0,id,pos_dt,cc_no,cc_txn_amt,count,pos_dt_index
0,0,1,2018-01-20,98397,0.009926,-0.366991,19
1,1,1,2018-02-17,196794,0.421319,1.115008,47
2,2,1,2018-05-13,98397,0.152435,-0.366991,132
3,3,1,2018-06-14,98397,0.152435,-0.366991,164
4,4,2,2018-01-04,9740,-0.073428,-0.366991,3


In [11]:
cc_weekly.head()

,Unnamed: 0,id,pos_week_index,cc_txn_amt,count
0,0,1,2,-0.048792,-0.584214
1,1,1,6,0.253095,-0.036353
2,2,1,18,0.055783,-0.584214
3,3,1,23,0.055783,-0.584214
4,4,2,0,-0.109958,-0.584214


In [12]:
train_cc_persons = cc_persons[cc_persons['id'] <= 50000]

In [13]:
train_cc_weekly = cc_weekly[cc_weekly['id'] <= 50000]

In [15]:
padding_value = cc_weekly_scaler.transform([[0., 0.]])
xs = create_weekly_credit(train_cc_weekly, padding_value)
ys = scaled_train_set.loc[train_cc_weekly['id'].drop_duplicates().to_numpy()].to_numpy()

In [ ]:
padding_value = float(-100)
xs = create_credit_seq(train_cc_persons, padding_value)
ys = scaled_train_set.loc[train_cc_persons['id'].drop_duplicates().to_numpy()].to_numpy()

In [16]:
xs.shape

(39950, 26, 2)

In [17]:
ys.shape

(39950, 1)

In [18]:
base_activation = 'tanh'
RNN = CuDNNLSTM
model = Sequential()
# model.add(Masking(padding_value))
model.add((RNN(64, return_sequences=True, input_shape=xs.shape[1:])))
model.add(Activation(base_activation))
model.add((RNN(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add((RNN(32, return_sequences=True)))
model.add(Activation(base_activation))
model.add((RNN(32, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add((RNN(16, )))
model.add(Activation(base_activation))
model.add(Dense(1, activation='linear'))

W1107 14:42:30.512545 140311903098688 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [30]:
model = Sequential()
model.add(Conv1D(64, kernel_size=3, strides=1, input_shape=xs.shape[1:], activation='relu'))
model.add(Conv1D(64, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPool1D())
model.add(Dropout(0.5))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPool1D())
model.add(Dropout(0.5))
model.add(Conv1D(4, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(4, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPool1D())
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(40))
# model.add(Dropout(0.25))
model.add(Dense(1))

In [31]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [32]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=opt )
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 24, 64)            448       
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 22, 64)            12352     
_________________________________________________________________
dropout_7 (Dropout)          (None, 22, 64)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 20, 16)            3088      
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 18, 16)            784       
_________________________________________________________________
dropout_8 (Dropout)          (None, 18, 16)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 16, 4)            

In [33]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [34]:
log_dir = 'cc-weekly_conv2'

model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                    TensorBoard(log_dir),
                    ModelCheckpoint(log_dir + '/ep{epoch}-val_SMAPE:{val_SMAPE}.h5')
                     ],
          initial_epoch=0,
         epochs=100)

Train on 31960 samples, validate on 7990 samples
Epoch 1/100
31960/31960 [==============================] - 5s 151us/step - loss: 0.1039 - val_loss: 0.0978
Epoch 1 | val-SMAPE: 90.21589753233148
Epoch 2/100
31960/31960 [==============================] - 4s 134us/step - loss: 0.1000 - val_loss: 0.0970
Epoch 2 | val-SMAPE: 90.30417639356531
Epoch 3/100
31960/31960 [==============================] - 4s 135us/step - loss: 0.0993 - val_loss: 0.0957
Epoch 3 | val-SMAPE: 90.43264704834534
Epoch 4/100
31960/31960 [==============================] - 4s 134us/step - loss: 0.0984 - val_loss: 0.0945
Epoch 4 | val-SMAPE: 90.54974373108305
Epoch 5/100
31960/31960 [==============================] - 4s 134us/step - loss: 0.0983 - val_loss: 0.0937
Epoch 5 | val-SMAPE: 90.62555740301839
Epoch 6/100
31960/31960 [==============================] - 4s 134us/step - loss: 0.0984 - val_loss: 0.0932
Epoch 6 | val-SMAPE: 90.68243895886985
Epoch 7/100
31960/31960 [==============================] - 4s 133us/step - 

KeyboardInterrupt: 

# KPLUS + Credit

In [11]:
kplus_n_threshold = 0
kplus_n_ids = scaled_kplus.groupby('id').size()
kplus_n_ids = set(kplus_n_ids[kplus_n_ids >= kplus_n_threshold].keys())
len(kplus_n_ids)

49269

In [12]:
cc_n_threshold = 0
cc_n_ids = cc_persons.groupby('id').size()
cc_n_ids = set(cc_n_ids[cc_n_ids >= cc_n_threshold].keys())
len(cc_n_ids)

51467

In [94]:
kplus_ids = set(scaled_kplus[scaled_kplus['id'] <= 50000]['id'].drop_duplicates())
cc_ids = set(cc_persons[cc_persons['id'] <= 50000]['id'].drop_duplicates())
print(len(kplus_ids), len(cc_ids), len(kplus_ids.union(cc_ids)), len(kplus_ids.intersection(cc_ids)))
ids = sorted(list(kplus_ids.union(cc_ids)))
# ids = sorted(list(kplus_ids.intersection(cc_ids, kplus_n_ids, cc_n_ids)))
ids_df = pd.DataFrame(ids, columns=['id'])

38056 39950 47174 30832


In [95]:
train_kplus = pd.merge(scaled_kplus, ids_df, on='id', how='right')
train_cc_persons = pd.merge(cc_persons, ids_df, on='id', how='right')
train_cc_weekly = pd.merge(cc_weekly, ids_df, on='id', how='right')

In [96]:
padding_value = float(-100)
kplus_padding = cc_padding = padding_value
kplus_padding = kplus_scaler.transform([[0,0]])
cc_padding = cc_scaler.transform([[0,0]])

kplus_xs = create_daily_kplus(train_kplus, kplus_padding)
cc_persons_xs = create_credit_seq(train_cc_persons, cc_padding)


KeyboardInterrupt: 

In [41]:
kplus_xs.shape, cc_weekly_xs.shape

((30832, 25, 2), (30832, 26, 2))

In [47]:

xs = np.concatenate((kplus_xs, cc_persons_xs), axis=2)
# xs = np.concatenate((np.expand_dims(kplus_xs[:, :, 0], axis=2), np.expand_dims(cc_persons_xs[:, :, 0], axis=2)), axis=2)
ys = scaled_train_set.loc[ids]['income'].to_numpy()

NameError: name 'cc_persons_xs' is not defined

In [97]:
padding_value = float(-100)
kplus_padding = cc_padding = padding_value
kplus_padding = kplus_scaler.transform([[0,0]])
cc_weekly_padding = cc_weekly_scaler.transform([[0,0]])

kplus_xs = create_weekly_kplus(train_kplus, kplus_padding)
cc_weekly_xs = create_weekly_credit(train_cc_weekly, cc_weekly_padding)

In [98]:
gap_padding = np.ones((len(kplus_xs), 1, 2), dtype=np.float32) * kplus_padding

xs = np.concatenate((np.concatenate((gap_padding, kplus_xs,), axis=1), cc_weekly_xs), axis=2)
# xs = np.concatenate((np.expand_dims(kplus_xs[:, :, 0], axis=2), np.expand_dims(cc_persons_xs[:, :, 0], axis=2)), axis=2)
ys = scaled_train_set.loc[ids]['income'].to_numpy()

In [99]:
xs.shape, ys.shape

((47174, 26, 4), (47174,))

In [100]:
base_activation = 'tanh'
RNN = CuDNNGRU
model = Sequential()
# model.add(Masking(padding_value, input_shape=xs.shape[1:] ))
model.add((RNN(24, return_sequences=True, input_shape=xs.shape[1:])))
model.add(Activation(base_activation))
model.add((RNN(24, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add((RNN(18, return_sequences=True, )))
model.add(Activation(base_activation))
model.add((RNN(18, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add((RNN(12, return_sequences=True, )))
model.add(Activation(base_activation))
model.add((RNN(12, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
# model.add(RNN(64, return_sequences=True))
# model.add(Activation(base_activation))
model.add(RNN(12, ))
model.add(Activation(base_activation))
# model.add(Dense(10))
# model.add(Dropout(0.25))


model.add(Dense(1, activation='linear'))

In [73]:
model = Sequential()
model.add(Conv1D(64, kernel_size=3, strides=1, input_shape=xs.shape[1:], activation='relu'))
model.add(Conv1D(64, kernel_size=3, strides=1, activation='relu'))
# model.add(Dropout(0.5))
model.add(Conv1D(32, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(32, kernel_size=3, strides=1, activation='relu'))
# model.add(Dropout(0.5))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(16, kernel_size=3, strides=1, activation='relu'))
# model.add(Dropout(0.5))
model.add(Conv1D(8, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(8, kernel_size=3, strides=1, activation='relu'))
# model.add(Dropout(0.5))
model.add(Conv1D(4, kernel_size=3, strides=1, activation='relu'))
model.add(Conv1D(4, kernel_size=3, strides=1, activation='relu'))
# model.add(MaxPool1D())
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(24))
# model.add(Dropout(0.25))
model.add(Dense(1))

In [101]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=Adam() )
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnngru_22 (CuDNNGRU)      (None, 26, 24)            2160      
_________________________________________________________________
activation_49 (Activation)   (None, 26, 24)            0         
_________________________________________________________________
cu_dnngru_23 (CuDNNGRU)      (None, 26, 24)            3600      
_________________________________________________________________
activation_50 (Activation)   (None, 26, 24)            0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 26, 24)            0         
_________________________________________________________________
cu_dnngru_24 (CuDNNGRU)      (None, 26, 18)            2376      
_________________________________________________________________
activation_51 (Activation)   (None, 26, 18)          

In [103]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [104]:
log_dir = 'logs/union_kplus_cc-weekly_gru3'

model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir),
                     ModelCheckpoint(log_dir + '/ep{epoch}-val_SMAPE:{val_SMAPE}.h5')
                     ],
          initial_epoch=0,
         epochs=100)

Train on 37739 samples, validate on 9435 samples
Epoch 1/100
37739/37739 [==============================] - 16s 429us/step - loss: 0.0960 - val_loss: 0.0901
Epoch 1 | val-SMAPE: 90.98825006081935
Epoch 2/100
37739/37739 [==============================] - 15s 398us/step - loss: 0.0902 - val_loss: 0.0893
Epoch 2 | val-SMAPE: 91.06826309902395
Epoch 3/100
37739/37739 [==============================] - 15s 396us/step - loss: 0.0879 - val_loss: 0.0859
Epoch 3 | val-SMAPE: 91.40838222512033
Epoch 4/100
37739/37739 [==============================] - 15s 401us/step - loss: 0.0869 - val_loss: 0.0858
Epoch 4 | val-SMAPE: 91.42273832785749
Epoch 5/100
37739/37739 [==============================] - 15s 385us/step - loss: 0.0865 - val_loss: 0.0844
Epoch 5 | val-SMAPE: 91.55572559729922
Epoch 6/100
37739/37739 [==============================] - 15s 394us/step - loss: 0.0856 - val_loss: 0.0843
Epoch 6 | val-SMAPE: 91.57238902643552
Epoch 7/100
37739/37739 [==============================] - 15s 397us/

KeyboardInterrupt: 

# DEMOGRAPHIC + CC + KPLUS

In [105]:

def create_all_x_data(ids):
    ids_df = pd.DataFrame(ids, columns=['id'])
    train_kplus = pd.merge(scaled_kplus, ids_df, on='id', how='right')
    train_cc_persons = pd.merge(cc_persons, ids_df, on='id', how='right')

    padding_value = float(-100)
    kplus_padding = cc_padding = padding_value
    kplus_padding = kplus_scaler.transform([[0,0]])
    cc_padding = cc_scaler.transform([[0,0]])
    kplus_xs = create_daily_kplus(train_kplus, kplus_padding)
    cc_persons_xs = create_credit_seq(train_cc_persons, cc_padding)
    transaction_xs = np.concatenate((kplus_xs, cc_persons_xs), axis=2)

    demographic_xs = create_demographic_data(raw_demographics.set_index('id').loc[ids])


    return demographic_xs, transaction_xs

In [106]:
def create_all_y_data(ids):
    return scaled_train_set.loc[ids]['income'].to_numpy()

In [16]:
ids = train_set['id'].to_list()
demographic_xs, transaction_xs = create_all_x_data(ids)
ys = create_all_y_data(ids)

In [17]:
demographic_xs.shape, transaction_xs.shape, ys.shape

((50000, 21), (50000, 181, 4), (50000,))

In [0]:
demo_x_train, demo_x_val, transaction_x_train, transaction_x_val, y_train, y_val \
= train_test_split(demographic_xs, transaction_xs, ys, test_size=0.2)

In [48]:
demo_x_train.shape

(40000, 21)

In [0]:
transaction_model = load_model('/content/logs/kplus_cc_conv_maxpool3/ep14-val_SMAPE:91.58859339541026.h5'
                               , custom_objects={'loss_func':SMAPE_loss(train_scaler)})

In [50]:
transaction_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 179, 64)           832       
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 177, 64)           12352     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 88, 64)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 88, 64)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 86, 16)            3088      
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 84, 16)            784       
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 42, 16)           

In [0]:
for layer in transaction_model.layers:
    layer.trainable = True

In [0]:
demographic_model = Sequential()
demographic_model.add(Dense(20, activation='relu', input_shape=(21,)))
demographic_model.add(Dense(20, activation='relu'))
demographic_model.add(Dense(10, activation='relu'))
demographic_model.add(Dense(1))

In [52]:
demographic_model.compile(optimizer='adam', loss=SMAPE_loss(train_scaler))
demographic_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 20)                440       
_________________________________________________________________
dense_16 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_17 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 1,081
Trainable params: 1,081
Non-trainable params: 0
_________________________________________________________________


In [53]:
demographic_model.fit(demo_x_train, y_train,
          validation_data=(demo_x_val, y_val),
          epochs=100)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 7s 171us/step - loss: 0.1056 - val_loss: 0.1045
Epoch 2/100
40000/40000 [==============================] - 6s 143us/step - loss: 0.1027 - val_loss: 0.1043
Epoch 3/100
40000/40000 [==============================] - 6s 144us/step - loss: 0.1027 - val_loss: 0.1036
Epoch 4/100
40000/40000 [==============================] - 6s 144us/step - loss: 0.1025 - val_loss: 0.1038
Epoch 5/100
40000/40000 [==============================] - 6s 145us/step - loss: 0.1024 - val_loss: 0.1037
Epoch 6/100
40000/40000 [==============================] - 6s 143us/step - loss: 0.1024 - val_loss: 0.1039
Epoch 7/100
40000/40000 [==============================] - 6s 144us/step - loss: 0.1024 - val_loss: 0.1039
Epoch 8/100
21440/40000 [===============>..............] - ETA: 2s - loss: 0.1016

KeyboardInterrupt: ignored

In [0]:
for layer in demographic_model.layers:
    layer.trainable = True

In [0]:
x = Concatenate()([demographic_model.layers[-2].output, transaction_model.layers[-2].output])
x = Dense(32,)(x)
x = Dense(16,)(x)
x = Dense(1)(x)
model = Model(inputs=[demographic_model.input, transaction_model.input], outputs=x)

In [19]:
model = load_model('logs/demo_kplus_cc_conv/ep18-val_SMAPE:91.62421650469304.h5',
                  custom_objects={'loss_func':SMAPE_loss(train_scaler)})

W1106 16:58:32.322900 140655518598976 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1375: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1106 16:58:33.236465 140655518598976 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [20]:
model.compile(optimizer='adam', loss=SMAPE_loss(train_scaler))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv1d_22_input (InputLayer)    (None, 181, 4)       0                                            
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 179, 64)      832         conv1d_22_input[0][0]            
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 177, 64)      12352       conv1d_22[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_13 (MaxPooling1D) (None, 88, 64)       0           conv1d_23[0][0]                  
____________________________________________________________________________________________

In [58]:
log_dir = 'logs/demo_kplus_cc_conv'

model.fit([demo_x_train, transaction_x_train], y_train,
          validation_data=([demo_x_val, transaction_x_val], y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir),
                     ModelCheckpoint(log_dir + '/ep{epoch}-val_SMAPE:{val_SMAPE}.h5')
                     ],
          initial_epoch=0,
         epochs=100)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 17s 434us/step - loss: 0.0893 - val_loss: 0.0909
Epoch 1 | val-SMAPE: 90.91143378138543
Epoch 2/100
40000/40000 [==============================] - 16s 401us/step - loss: 0.0898 - val_loss: 0.0866
Epoch 2 | val-SMAPE: 91.3358620095253
Epoch 3/100
40000/40000 [==============================] - 16s 400us/step - loss: 0.0872 - val_loss: 0.0863
Epoch 3 | val-SMAPE: 91.36757788538932
Epoch 4/100
40000/40000 [==============================] - 16s 400us/step - loss: 0.0869 - val_loss: 0.0864
Epoch 4 | val-SMAPE: 91.36087724685669
Epoch 5/100
40000/40000 [==============================] - 16s 401us/step - loss: 0.0856 - val_loss: 0.0913
Epoch 5 | val-SMAPE: 90.86773957848548
Epoch 6/100
40000/40000 [==============================] - 16s 399us/step - loss: 0.0856 - val_loss: 0.0863
Epoch 6 | val-SMAPE: 91.36810836315155
Epoch 7/100
40000/40000 [==============================] - 16s 399us/

KeyboardInterrupt: ignored

In [23]:
log_dir = 'logs/demo_kplus_cc_conv_no-val'

model.fit([demographic_xs, transaction_xs], ys,
#           validation_data=([demo_x_val, transaction_x_val], y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[
#               EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir),
                     ModelCheckpoint(log_dir + '/ep{epoch}-loss:{loss}.h5')
                     ],
          initial_epoch=0,
         epochs=100)

Epoch 1/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.0848
Epoch 2/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.0842
Epoch 3/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.0843
Epoch 4/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.0843
Epoch 5/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.0839
Epoch 6/100
50000/50000 [==============================] - 8s 163us/step - loss: 0.0847
Epoch 7/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.0841
Epoch 8/100
50000/50000 [==============================] - 8s 163us/step - loss: 0.0840
Epoch 9/100
50000/50000 [==============================] - 8s 162us/step - loss: 0.0844
Epoch 10/100
50000/50000 [==============================] - 8s 163us/step - loss: 0.0838
Epoch 11/100
50000/50000 [==============================] - 8s 163us/step - loss: 0.0840
Epoch 12/100
 9952/50000 [====

KeyboardInterrupt: 

In [24]:
demo_test_xs, transaction_test_xs = create_all_x_data(test_set['id'].to_numpy())

In [25]:
ys_pred = model.predict([demo_test_xs, transaction_test_xs])

In [26]:
test_incomes = train_scaler.inverse_transform(ys_pred)

In [27]:
answer = test_set.copy()
answer['income'] = test_incomes
answer.to_csv(log_dir+'/answer.csv')